In [1]:
%matplotlib inline

In [2]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torch.optim import lr_scheduler
import pandas as pd
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from efficientnet_pytorch import EfficientNet
import matplotlib.pyplot as plt
import time
import os
import copy
import shutil

!jupyter nbextension enable --py widgetsnbextension

plt.ion()

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [3]:
for num in range(18):
    print(num, len(os.listdir('/opt/ml/input/ensemble/data_1/train/'+str(num))))

0 4836
1 3238
2 2110
3 4770
4 4626
5 1554
6 969
7 649
8 433
9 954
10 928
11 307
12 970
13 666
14 413
15 955
16 917
17 309


In [4]:
# https://pytorch.org/vision/stable/transforms.html

data_transforms = {
    'train': transforms.Compose([
        transforms.CenterCrop(384),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.CenterCrop(384),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

data_dir = '/opt/ml/input/ensemble/data_1'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'val':
                scheduler.step()

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model_ft, "/opt/ml/model/model_data_1.pt")

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

In [6]:
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes=18, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [7]:
# https://pytorch.org/vision/stable/models.html

# model_ft = models.resnet18(pretrained=True)
model_ft = models.resnet34(pretrained=True)
# model_ft = models.resnet50(pretrained=True)
# model_ft = models.resnet101(pretrained=True)
# model_ft = models.resnet152(pretrained=True)
# model_ft = models.resnext50_32x4d(pretrained=True)
# model_ft = models.resnext101_32x8d(pretrained=True)
# model_ft = models.wide_resnet50_2(pretrained=True)
# model_ft = models.wide_resnet101_2(pretrained=True)
# model_ft = EfficientNet.from_pretrained('efficientnet-b0', num_classes=18)

num_ftrs = model_ft.fc.in_features

model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)

criterion = LabelSmoothingLoss()

optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.0001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.9)

In [8]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=15)

Epoch 0/14
----------
train Loss: 0.9895 Acc: 0.6913
val Loss: 0.4645 Acc: 0.8233

Epoch 1/14
----------
train Loss: 0.4615 Acc: 0.8465
val Loss: 0.3346 Acc: 0.8724

Epoch 2/14
----------
train Loss: 0.3301 Acc: 0.8888
val Loss: 0.2732 Acc: 0.8866

Epoch 3/14
----------
train Loss: 0.2412 Acc: 0.9200
val Loss: 0.1733 Acc: 0.9330

Epoch 4/14
----------
train Loss: 0.1789 Acc: 0.9431
val Loss: 0.1985 Acc: 0.9204

Epoch 5/14
----------
train Loss: 0.1250 Acc: 0.9622
val Loss: 0.1160 Acc: 0.9562

Epoch 6/14
----------
train Loss: 0.0964 Acc: 0.9723
val Loss: 0.0878 Acc: 0.9694

Epoch 7/14
----------
train Loss: 0.0731 Acc: 0.9789
val Loss: 0.0789 Acc: 0.9731

Epoch 8/14
----------
train Loss: 0.0558 Acc: 0.9850
val Loss: 0.0695 Acc: 0.9763

Epoch 9/14
----------
train Loss: 0.0454 Acc: 0.9882
val Loss: 0.0650 Acc: 0.9778

Epoch 10/14
----------
train Loss: 0.0363 Acc: 0.9913
val Loss: 0.0554 Acc: 0.9821

Epoch 11/14
----------
train Loss: 0.0307 Acc: 0.9921
val Loss: 0.0496 Acc: 0.9858

Ep